# Model to ICEWS Events

In this analysis, I'm going to build a basic BDM model for the world based on COW alliance taus (with position = similarity to United States, to capture unipolarity) and see how predicted conflicts align with ICEWS events.

In [1]:
import copy
from collections import defaultdict, namedtuple, Counter
from itertools import combinations, permutations


import numpy as np
import scipy.stats

import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from 

In [ ]:
from negotiation_model import *
from bdm_agent import *
from balancing_agent import *


In [ ]:
class BDMActor(NegotiationActor):
    DecisionClass = Balancing_Agent

In [ ]:
NegotiationModel.step_stages = ["initialize", "send_threats", 
                                "resolve_threats",  "finalize", 
                                "resolve_attacks"]

## Load Alliance Data

In [ ]:
start_year = 2004

In [ ]:
alliance_dyads = pd.read_csv("/Users/dmasad/Data/COW/Alliances/alliance_v4.1_by_dyad_yearly.csv")
alliance_dyads = alliance_dyads[alliance_dyads.year==start_year]
alliance_dyads["Alliance"] = 0
alliance_dyads.loc[(alliance_dyads.entente==1),"Alliance"] = 1
alliance_dyads.loc[(alliance_dyads.nonaggression==1),"Alliance"] = 2
alliance_dyads.loc[(alliance_dyads.neutrality==1),"Alliance"] = 3
alliance_dyads.loc[(alliance_dyads.defense==1),"Alliance"] = 4

In [ ]:
ccode_to_name = {}
for key, val in alliance_dyads.groupby(["ccode1", "state_name1"]):
    ccode, name = key
    ccode_to_name[ccode] = name
for key, val in alliance_dyads.groupby(["ccode2", "state_name2"]):
    ccode, name = key
    ccode_to_name[ccode] = name

In [ ]:
G = nx.Graph()
for i, row in alliance_dyads.iterrows():
    G.add_edge(row.state_name1, row.state_name2, weight=row.Alliance)

In [ ]:
pos = nx.spring_layout(G, k=0.15, scale=8)

fig, ax = plt.subplots(figsize=(12, 12))
ax.axis('off')

#nx.draw_networkx_nodes(G, pos=pos, node_size=100)
nx.draw_networkx_edges(G, pos=pos, alpha=0.5)
nx.draw_networkx_labels(G, pos=pos)

ax.set_xlim(-0.25, 8.25)
ax.set_ylim(-0.25, 8.25)

In [ ]:
names = {node: i for i, node in enumerate(G.nodes())}
mat = nx.to_numpy_matrix(G)

def get_tau(state_1, state_2, p_val=False):
    try:
        n1 = names[state_1]
        n2 = names[state_2]
        v1 = mat[n1]
        v2 = mat[n2]
        tau = scipy.stats.kendalltau(v1, v2)
    except:
        tau = (0, 0)
    if p_val:
        return tau
    else:
        return tau[0]

In [ ]:
# How to normalize positions
positions = [get_tau("United States of America", actor)
             for actor in G.nodes()]
max_pos = max(positions)
min_pos = min(positions)
print(max_pos)
print(min_pos)

## NMC Data

In [ ]:
nmc = pd.read_csv("/Users/dmasad/Data/COW/NMC_v4_0.csv")
nmc = nmc[nmc.year==start_year].copy()

In [ ]:
# Add full names
system_members = pd.read_csv("/Users/dmasad/Data/COW/SystemMembership2011/states2011.csv")
system_members = system_members.drop_duplicates("ccode")
nmc = nmc.merge(system_members[["ccode", "statenme"]], how='left', on="ccode")

In [ ]:
for node in G.nodes():
    if len(nmc[nmc.statenme==node]) == 0:
        print(node)

In [ ]:
nmc[nmc.statenme=="Germany"]

# Build Model

In [ ]:
# Build a dictionary of Actor data
ActorData = namedtuple("ActorData", ["name", "position", "capability"])
actor_data = defaultdict(ActorData)

In [ ]:
for actor in G.nodes():
    position = get_tau("United States of America", actor, p_val=False)
    position = (position + abs(min_pos))/(max_pos - min_pos)
    name = actor if actor != "German Federal Republic" else "Germany"
    capability = nmc[nmc.statenme==name].cinc.values[0]
    actor_data[name] = ActorData(name, position, capability)

### Single Model Run

In [ ]:
agents = []
for actor in actor_data.values():
    new_agent = BDMActor(actor.name, actor.capability, actor.position, 1)
    agents.append(new_agent)

model = Real_Negotiation_Model(agents)

In [ ]:
for i in range(25):
    for agent in model.agents:
            agent.salience = random.random()
    model.step()

df = model.datacollector.get_model_vars_dataframe()
fig, ax = plt.subplots(figsize=(12,4))
df.plot(ax=ax)
ax.set_ylim(0, 1)

### Top 50 agents

In [ ]:
top_actors = nmc.sort("cinc", ascending=False).head(50)["statenme"].unique()

In [ ]:
top_actors

In [ ]:
top_actor_dicts = []
for key in top_actors:
    if key in actor_data:
        a = actor_data[key]
        top_actor_dicts.append({"Name": a.name, 
                                "Position": a.position,
                                "Capability": a.capability})
    else:
        top_actor_dicts.append({"Name": key, 
                                "Position": None,
                                "Capability": nmc[nmc.statenme==key].cinc.values[0]})
actor_df = pd.DataFrame(top_actor_dicts)
actor_df = actor_df[["Name", "Position", "Capability"]]
actor_df.sort("Name", inplace=True)

In [ ]:
print(actor_df.to_latex(float_format=lambda x: "{:.3f}".format(x)))

In [ ]:
agents = []
for name in top_actors:
    if name in actor_data:
        actor = actor_data[name]
        new_agent = BDMActor(actor.name, actor.capability, actor.position, 1)
        new_agent.decision_model.Q = 0.5
        agents.append(new_agent)
    else:
        print(name)

#model = Real_Negotiation_Model(agents)
model = NegotiationModel(agents)

In [ ]:
for i in range(24):
    for agent in model.agents:
            agent.salience = random.random()
    model.step()

df = model.datacollector.get_model_vars_dataframe()
fig, ax = plt.subplots(figsize=(12,4))
df.plot(ax=ax)
ax.set_ylim(0, 1)

In [ ]:
model.schedule.steps

In [ ]:
model.log.get_events(timestamp=20, action="Attack")

### Multiple iterations

In [ ]:
agents = []
for name in top_actors:
    if name in actor_data:
        actor = actor_data[name]
        new_agent = BDMActor(actor.name, actor.capability, actor.position, 1)
    else:
        capability = nmc[nmc.statenme==name].cinc.values[0]
        pos = 0.5 + random.normalvariate(0, 0.1)
        new_agent = BDMActor(name, capability, 0.5, 1)

    new_agent.decision_model.Q = 0.5
    new_agent.decision_model.T = 0.5
    agents.append(new_agent)
    
model = NegotiationModel(agents)

In [ ]:
all_models = []
for i in range(100):
    new_model = copy.deepcopy(model)
    for agent in new_model.agents:
            agent.salience = random.random()
    # Run the model:
    for j in range(24):
        new_model.step()
        if all((abs(agent.position - new_model.agents[0].position)<0.05) 
               for agent in new_model.agents):
            break
    
    all_models.append(new_model)
    print(i)

In [ ]:
df = all_models[-1].datacollector.get_model_vars_dataframe()
fig, ax = plt.subplots(figsize=(12,4))
df.plot(ax=ax)
ax.set_ylim(0, 1)

In [ ]:
conflict_dyads = Counter()
conflict_models = defaultdict(set)

for i, model in enumerate(all_models):
    for event in model.log.get_events(action="Attack"):
        source = event.source
        target = event.target
        dyad = (source, target)
        conflict_dyads[dyad] += 1
        conflict_models[dyad].add(i)

conflict_models = {k: len(v) for k, v in conflict_models.items()}        
        
# Fill in zeroes
for source, target in permutations(top_actors, 2):
    dyad = (source, target)
    if dyad not in conflict_dyads:
        conflict_dyads[dyad] = 0
    if dyad not in conflict_models:
        conflict_models[dyad] = 0

## Get ICEWS Data

Data prepared in [6.05 ICEWS Aggregation](http://localhost:8888/notebooks/Programming/ConflictModel/negotiation_model/6.05%20ICEWS%20Aggregation.ipynb)

In [ ]:
icews = pd.read_csv("Negative_Dyads_2005-2006.csv")
icews = icews[["Source_Country", "Target_Country", "Event_ID"]]
icews.rename(columns={"Event_ID": "Count"}, inplace=True)

In [ ]:
icews.head()

In [ ]:
icews.replace("Democratic Republic of Congo", "Democratic Republic of the Congo", inplace=True)
icews.replace("Russian Federation", "Russia", inplace=True)
icews.replace("United States", "United States of America", inplace=True)

# Compare

In [ ]:
model_conflicts = pd.DataFrame({"Model_Conflicts": dict(conflict_dyads),
                               "Models_with_Conflicts": conflict_models})
model_conflicts.reset_index(inplace=True)
model_conflicts.rename(columns={"level_0": "Source_Country",
                               "level_1": "Target_Country"}, inplace=True)

In [ ]:
model_conflicts.head()

In [ ]:
df = model_conflicts.merge(icews, how="left", on=["Source_Country", "Target_Country"])
df.rename(columns={"Count": "ICEWS_Count"}, inplace=True)
df.fillna(0, inplace=True)

In [ ]:
df.head()

In [ ]:
import statsmodels.api as sm
import statsmodels
from statsmodels.discrete.discrete_model import NegativeBinomial
from statsmodels.discrete.discrete_model import Poisson
from statsmodels.miscmodels.count import PoissonZiGMLE

In [ ]:
df["Const."] = 1

In [ ]:
lm = sm.OLS(df["ICEWS_Count"], df[["Const.", "Model_Conflicts"]])
fit = lm.fit()

In [ ]:
print(fit.summary())

In [ ]:
nb = NegativeBinomial(df["ICEWS_Count"], df[["Const.", "Model_Conflicts"]])
fit = nb.fit()
print(fit.summary())

In [ ]:
ps = Poisson(df["ICEWS_Count"], df[["Const.", "Model_Conflicts"]])
fit = ps.fit_regularized()
print(fit.summary())

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.scatter(df.Model_Conflicts, df.ICEWS_Count)
ax.set_xlabel("Model Conflicts")
ax.set_ylabel("ICEWS Conflicts")
#ax.set_xlim(-5, 100)

In [ ]:
df.ICEWS_Count.hist()
plt.yscale('log')

In [ ]:
df.corr(method="spearman")

In [ ]:
df.corr()

### At least N events

In [ ]:
df["ICEWS_Dummy"] = 0
df["ICEWS_Dummy"][df.ICEWS_Count > 5] = 1

In [ ]:
logit = sm.Logit(df["ICEWS_Dummy"], df[["Const.", "Model_Conflicts"]])
fit = logit.fit()
print(fit.summary())

In [ ]:
pd.crosstab(df.ICEWS_Dummy, df.Model_Conflicts)

In [ ]:
logit = sm.Logit(df["ICEWS_Dummy"], df[["Const.", "Models_with_Conflicts"]])
fit = logit.fit()
print(fit.summary())

In [ ]:
df["Model_Dummy"] = 0
df["Model_Dummy"][df.Model_Conflicts > 0] = 1

In [ ]:
logit = sm.Logit(df["ICEWS_Dummy"], df[["Const.", "Model_Dummy"]])
fit = logit.fit()
print(fit.summary())

In [ ]:
df["Model_Dummy"] = 0
df["Model_Dummy"][df.Models_with_Conflicts > 0] = 1

logit = sm.Logit(df["ICEWS_Dummy"], df[["Const.", "Model_Dummy"]])
fit = logit.fit()
print(fit.summary())

### Is it better than just using $\Delta$Taus?

In [ ]:
dyad_taus = {}
for i, row in df.iterrows():
    source = row.Source_Country
    target = row.Target_Country
    tau = get_tau(source, target)
    dyad_taus[(source, target)] = tau

In [ ]:
taus = pd.Series(dyad_taus).reset_index()
taus.rename(columns={"level_0": "Source_Country", 
                     "level_1": "Target_Country",
                     0: "Tau"}, inplace=True)

In [ ]:
df = df.merge(taus, how='left', on=["Source_Country", "Target_Country"])

In [ ]:
df["Tau_Dummy"] = 0
df["Tau_Dummy"][df.Tau != 0] = 1

In [ ]:
lm = sm.OLS(df["ICEWS_Count"], df[["Const.", "Tau"]])
fit = lm.fit()
print(fit.summary())

In [ ]:
logit = sm.Logit(df["ICEWS_Dummy"], df[["Const.", "Tau_Dummy", "Tau"]])
fit = logit.fit()
print(fit.summary())

In [ ]:
logit = sm.Logit(df["ICEWS_Dummy"], df[["Const.", "Tau", "Model_Conflicts"]])
fit = logit.fit()
print(fit.summary())

In [ ]:
ps = Poisson(df["ICEWS_Count"], df[["Const.", "Tau", "Model_Conflicts"]])
fit = ps.fit_regularized()
print(fit.summary())